# 3.2 데이터 불러오기 및 전처리

### 3.2.1 불러올 데이터의 형태 파악

In [ ]:
# 예제 3-1 kto_201901.xlsx 엑셀 데이터 불러오기
import pandas as pd 
kto_201901 = pd.read_excel('./files/kto_201901.xlsx', 
                          header=1,
                          usecols='A:G',
                          skipfooter=4)
kto_201901.head()

In [ ]:
# 예제 3-2 tail() 함수 활용하기 
kto_201901.tail()

### 3.2.3. 데이터 전처리 하기

In [ ]:
# 예제 3-3 info() 함수 활용하기 
kto_201901.info()

In [ ]:
# 예제 3-4 describe() 함수 활용하기 
kto_201901.describe()

In [ ]:
# 예제 3-5 각 컬럼에서 0인 부분 필터링하기 
condition = (kto_201901['관광'] == 0) \
                | (kto_201901['상용'] == 0) \
                | (kto_201901['공용'] == 0) \
                | (kto_201901['유학/연수'] == 0) 
kto_201901[condition]

In [ ]:
# 예제 3-6 기준년월 컬럼 생성하기 
kto_201901['기준년월'] = '2019-01'
kto_201901.head()

In [ ]:
# 예제 3-7  unique() 함수 활용하기 
kto_201901['국적'].unique()

In [ ]:
# 예제 3-8 대륙목록 만들기 
continents_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']
continents_list

In [ ]:
# 예제 3-9 대륙목록에 해당하는 값 제외하기 
condition = (kto_201901.국적.isin(continents_list) == False) 
kto_201901_country = kto_201901[condition]
kto_201901_country['국적'].unique()

In [ ]:
# 예제 3-10 kto_201901_country 데이터 head() 함수로 확인하기 
kto_201901_country.head()

In [ ]:
# 예제 3-11  index 재설정하기 
kto_201901_country_newindex = kto_201901_country.reset_index(drop=True) 
kto_201901_country_newindex.head()

In [ ]:
# 예제 3-12 대륙 컬럼값 만들기
continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 \
+ ['아프리카']*2 + ['기타대륙'] + ['교포']
print(continents)

In [ ]:
# 예제 3-13 대륙 컬럼 생성하기 
kto_201901_country_newindex['대륙'] = continents
kto_201901_country_newindex.head()

In [ ]:
# 예제 3-14 kto_201901_country_newindex 데이터 tail() 확인하기 
kto_201901_country_newindex.tail()

In [ ]:
# 예제 3-15 관광객비율(%) 컬럼 생성하기 
kto_201901_country_newindex['관광객비율(%)'] = \
round(kto_201901_country_newindex['관광'] / kto_201901_country_newindex['계'] * 100, 1)
kto_201901_country_newindex.head()

In [ ]:
# 예제 3-16 관광객비율(%) 컬럼으로 내림차순 정렬하기  
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=False).head()

In [ ]:
# 예제 3-17 관광객비율(%) 컬럼으로 오름차순 정렬하기
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=True).head()

In [ ]:
# 예제 3-18 pivot_table() 함수 활용하기 
kto_201901_country_newindex.pivot_table(values = '관광객비율(%)',
                                       index = '대륙',
                                       aggfunc = 'mean')

In [ ]:
# 예제 3-19 중국 국적만 필터링하기 
condition = (kto_201901_country_newindex.국적 == '중국')
kto_201901_country_newindex[condition]

In [ ]:
# 예제 3-20 2019년 1월 우리나라 방문하는 전체 외국인 관광객 숫자 구하기 
tourist_sum = sum(kto_201901_country_newindex['관광'])
tourist_sum 

In [ ]:
# 예제 3-21 전체비율(%) 컬럼 생성하기
kto_201901_country_newindex['전체비율(%)'] = \
round(kto_201901_country_newindex['관광'] / tourist_sum * 100, 1)
kto_201901_country_newindex.head()

In [ ]:
# 예제 3-22 전체비율(%) 컬럼 기준으로 내림차순 정렬하기 
kto_201901_country_newindex.sort_values('전체비율(%)', ascending=False).head()

### 2.2.4. 데이터 전처리 과정 함수로 만들기 

In [ ]:
#예제 2-23 create_kto_data 함수 생성하기 
def create_kto_data(yy, mm):  
    #1. 불러올 Excel 파일 경로를 지정해주기
    file_path = './files/kto_{}{}.xlsx'.format(yy, mm)  
    
    # 2. Excel 파일 불러오기 
    df = pd.read_excel(file_path, header=1, skipfooter=4, usecols='A:G')
    
    # 3. "기준년월" 컬럼 추가하기 
    df['기준년월'] = '{}-{}'.format(yy, mm) 
    
    # 4. "국적" 컬럼에서 대륙 제거하고 국가만 남기기 
    ignore_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']    # 제거할 대륙명 선정하기 
    condition = (df['국적'].isin(ignore_list) == False)    # 대륙 미포함 조건 
    df_country = df[condition].reset_index(drop=True) 
    
    # 5. "대륙" 컬럼 추가하기 
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['대양주']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']    # 대륙 컬럼 생성을 위한 목록 만들어 놓기 
    df_country['대륙'] = continents   
                       
    # 6. 국가별 "관광객비율(%)" 컬럼 추가하기 
    df_country['관광객비율(%)'] = round(df_country.관광 / df_country.계 * 100, 1) 
                       
    # 7. "전체비율(%)" 컬럼 추가하기 
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광'] / tourist_sum * 100, 1)
    
    # 8. 결과 출력하기 
    return(df_country)


In [ ]:
# 예제 2-24 create_kto_data() 함수를 활용해서 2018년 12월 데이터 불러오기  
kto_test = create_kto_data(2018, 12)
kto_test.head()

### 3.2.5. 반복문(for문)을 통해 다수의 엑셀 데이터 불러와서 합치기 

In [ ]:
# 예제 3-25 이중 반복문으로 기준년월 출력하기 
for yy in range(2010, 2020): 
    for mm in range(1, 13): 
        yymm = '{}{}'.format(yy, mm)
        print(yymm)

In [ ]:
# 예제 3-26 숫자 1 출력하기 
mm = 1 
print(mm) 

In [ ]:
# 예제 3-27 zfill() 함수 이용하기 1(0을 포함한 두 자리로 출력하기)
print(str(mm).zfill(2))

In [ ]:
# 예제 3-28 zfill() 함수 이용하기 1(0을 포함한 3자리, 4자리로 출력하기)
print(str(mm).zfill(3))
print(str(mm).zfill(4))

In [ ]:
# 예제 3-29 6자리로 정렬하여 기준년월 출력하기 
for yy in range(2010, 2020):
    for mm in range(1, 13):
        mm_str = str(mm).zfill(2)
        yymm = '{}{}'.format(yy, mm_str)
        print(yymm)

In [ ]:
# 예제 3-30 데이터를 담을 빈 데이터프레임 만들기 
df = pd.DataFrame()

In [ ]:
# 예제 3-31 패턴 분석결과를 이중 반복문을 통해 구현
for yy in range(2010, 2020):
    for mm in range(1, 13):
        temp = create_kto_data(str(yy), str(mm).zfill(2))
        df = df.append(temp, ignore_index=True)

In [ ]:
# 예제 3-32 df 데이터 head() 함수로 확인하기
df.head()

In [ ]:
# 예제 3-33 df 데이터 tail() 함수로 확인하기
df.tail()

In [ ]:
# 예제 3-34 통합 데이터 생성하기 
df = pd.DataFrame()

# 패턴 분석결과를 이중 반복문을 통해 구현 
for yy in range(2010, 2020):
    for mm in range(1, 13):
        # try 이하 구문을 실행 
        try:
            # 앞서 만든 create_kto_data 함수로 Excel 데이터를 temp 에 불러오기 
            temp = create_kto_data(str(yy), str(mm).zfill(2))

            # temp 로 불러온 데이터를 df 밑으로 추가하기 
            df = df.append(temp, ignore_index=True)
        # 만약 try 구문에서 에러가 발생하면 except 이후를 실행 
        except:
            pass

In [ ]:
# 예제 3-35 df 데이터 info() 함수로 확인하기 
df.info()

In [ ]:
# 예제 3-36 df 데이터 엑셀로 저장하기
df.to_excel('./files/kto_total.xlsx', index = False)